This are the task we  want to do
* Get website page
* Scrap the Content
* Build email body/content
* Email authetication
* Sent email to the require people
Popular Python Libraries for Web Scraping

* Requests
* Beautiful Soup 4
* lxml
* Selenium
* Scrapy

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests     # for requesting for the HTTP
import bs4          # this should be used  when importing beautiful soup
from bs4 import BeautifulSoup     # for webscrapping i.e extract content from website
import smtplib            # for email authetication
import email.mime          # this is for creating the email body or content
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email import message
import datetime            # this is for manipulating and accessing date and time
from selenium import webdriver  # for controlling web browsers through programs and performing browser automation
import scrapy
from urllib.request import urlopen
from apscheduler.schedulers.background import BackgroundScheduler  # for scheduling 
%matplotlib inline

In [3]:
now=datetime.datetime.now()  # This  uses is the current datetime i.e system time
content= ''                  # Email Content placeholder

In [4]:
#print(now)   ## Checking the datetime now

2022-01-05 10:29:18.507559


# 1 Scrapping Google news

In [5]:
def scrap(url):
    Client = urlopen(url)
    xml_page = Client.read()
    Client.close()
    soup_page = BeautifulSoup(xml_page, "xml")
    news_list = soup_page.findAll("item")

    # print news title, url and publish date
    
    for news in news_list:
        news_list = set()
        print(news.title.text), print(news.link.text), print(news.pubDate.text)

In [6]:
#scrap("http://news.google.com/news/rss")
scrap("https://jira.customappsteam.co.uk/issues/?filter=-4&jql=Sprint%20%3D%206476%20order%20by%20created%20DESC")

URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

# 2 Scrapping weather Data

In [6]:
import requests
page = requests.get("https://www.timeanddate.com/weather/india/chennai/historic")

In [7]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(page.content,"html.parser")

In [8]:
table=soup.find_all("table",{"class":"zebra tb-wt fw va-m tb-hover"})
National=[]
for i,items in enumerate(table):
    for i,row in enumerate(items.find_all("tr")):
        d = {}
        try:
#             print(i , row.find_all("td",{"class":""})[0].text)
            d['Temp'] = row.find_all("td",{"class":""})[0].text
        except:
            d['Temp'] = np.nan
            
        try:
#             print(i , row.find("td",{"class":"small"}).text)
            d['Weather'] = row.find("td",{"class":"small"}).text
        except:
            d['Weather']= np.nan
            
        try:   
#             print(i , row.find_all("td",{"class":"sep"})[0].text)
            d['Wind'] = row.find_all("td",{"class":"sep"})[0].text
        except:
            d['Wind'] = np.nan
            
        try:  
#            print(i, row.find("span",{"class":"comp sa16"})['title'])
            d['Direction'] = row.find("span")["title"]
        except:
            try:
                d['Direction'] = row.find("span",{"class":"comp sa16"})["title"]
            except:
                d['Direction'] = np.nan
            
        try:
#             print(i , row.find_all("td",{"class":""})[2].text)
            d['Humidity'] = row.find_all("td",{"class":""})[2].text
        except:
            d['Humidity'] = np.nan
        
        try:
#             print(i , row.find_all("td",{"class":"sep"})[1].text)
            d['Barometer'] =  row.find_all("td",{"class":"sep"})[1].text
        except:
            d['Barometer'] = np.nan
    
        try:
#             print(i , row.find_all("td",{"class":""})[3].text)
            d['Visibility'] =  row.find_all("td",{"class":""})[3].text
        except:
             d['Visibility'] = np.nan
                
        National.append(d)


In [9]:
import pandas
df = pandas.DataFrame(National)

In [10]:
df2 = df.dropna(how = 'all')
df2 = df2.reset_index()
df2.pop('index')
df2['Barometer'] = df2['Barometer'].str.extract('(\d+\.\d+)') + r'"Hg'
df2['Weather'] = df2['Weather'].str.replace(" ","")
df2['Visibility'] = df2['Visibility'].str.extract('(\d+)') + 'mi'
df2['Wind'] = df2['Wind'].str.extract('(\d+)') + 'mph'
df2['Temp'] = df2['Temp'].str.extract('(\d+)') + u'\N{DEGREE SIGN}' + 'F'

In [11]:
df2.to_csv('data.csv', columns = ['Temp', 'Weather', 'Wind', 'Direction', 'Humidity', 'Barometer',
       'Visibility'])

In [12]:
df2

,Temp,Weather,Wind,Direction,Humidity,Barometer,Visibility
0,27°F,Fog.,NaN,Wind blowing from 0° North to South,79%,NaN,4mi
1,27°F,Fog.,NaN,Wind blowing from 0° North to South,79%,NaN,4mi
2,27°F,Fog.,NaN,Wind blowing from 0° North to South,79%,NaN,4mi
3,27°F,Haze.,NaN,Wind blowing from 0° North to South,74%,NaN,5mi
4,28°F,Haze.,3mph,Wind blowing from 120° East-southeast to West-...,70%,NaN,5mi
5,27°F,Haze.,3mph,Wind blowing from 110° East-southeast to West-...,74%,NaN,5mi
6,28°F,Haze.,3mph,Wind blowing from 100° East to West,70%,NaN,5mi
7,28°F,Haze.,3mph,Wind blowing from 100° East to West,70%,NaN,3mi
8,28°F,Haze.,5mph,Wind blowing from 110° East-southeast to West-...,74%,NaN,3mi
9,29°F,Haze.,6mph,Wind blowing from 110° East-southeast to West-...,66%,NaN,3mi


# Simple sending message from email

In [18]:
from_addr ='wilsonadejo@yahoo.com'
to_addr = 'wilsonadejo@outlook.com'
subject = 'Becoming guru'
body = 'All hail the very powerful python guru. You are  doing well my guy'

msg = message.Message()
msg.add_header('from',from_addr)
msg.add_header('to', to_addr)
msg.add_header('subject', subject)
msg.set_payload(body)

In [19]:
msg

# Using Autoscraper to scap  website

In [49]:
from autoscraper import AutoScraper

In [55]:
# Define the URL and what you want there
url = 'https://www.analyticsvidhya.com/blog/category/machine-learning/'
wanted_list = ['Confusion Matrix: Detailed intuition and trick to learn']

In [56]:
#Initiate the AutoScraper
scraper=AutoScraper()

In [57]:
# Building the Object  and  show the result
result=scraper.build(url, wanted_list)
print(result)

[]


In [58]:
# To save the model

scraper.save('blogs')      #Give it a file path

In [60]:
# To load the model
scraper.load('blogs')

# 3 Feature Engineering

https://medium.com/@engelanalytics/feature-engineering-in-python-2fdb9bb8ee7a
https://github.com/rasgointelligence/feature-engineering-tutorials/tree/main/feature-transformation

In [15]:
df = pd.DataFrame({'A': ['text']*20,
                   'B': [1, 2.2, 3, 4]*5,
                   'C': [True, False]*10,
                   'D': pd.to_datetime('2020-01-01')
                  })

In [16]:
df

,A,B,C,D
0,text,1.0,True,2020-01-01
1,text,2.2,False,2020-01-01
2,text,3.0,True,2020-01-01
3,text,4.0,False,2020-01-01
4,text,1.0,True,2020-01-01
5,text,2.2,False,2020-01-01
6,text,3.0,True,2020-01-01
7,text,4.0,False,2020-01-01
8,text,1.0,True,2020-01-01
9,text,2.2,False,2020-01-01
